# Metrics, Metrics, Metrics

When choosing hyperparameters, you want to select those that make the models you design more accurate. In machine learning, a metric is a function that gauges the accuracy of predictions from a trained model. Hyperparameter optimization is done to optimize for hyperparameters that maximize (or minimize) this metric on the validation set. While this sounds simple up front, the notion of accuracy can in fact be quite subtle. Suppose you have a binary classifier. Is it more important to never mislabel false samples as true or to never mislabel true samples as false? How can you choose for model hyperparameters that satisfy the needs of your applications?

The answer turns out to be to choose the correct metric. In this section, we will discuss many different metrics for classification and regression problems. We will comment on the qualities each metric emphasizes. There is no best metric, but there are more suitable and less suitable metrics for different applications.

<div style="border: 1px solid black; padding: 10px;"><b style="font-size: 2em;">Metrics Aren’t a Replacement for Common Sense!</b><br> Metrics are terribly blind. They only optimize for a single quantity. Consequently, blind optimization of metrics can lead to entirely unsuitable outcomes. On the web, media sites often choose to optimize the metric of “user clicks.” Some enterprising young journalist or advertiser then realized that titles like “You’ll never believe what happened when X” induced users to click at higher fractions. Lo and behold, clickbait was born. While clickbait headlines do indeed induce readers to click, they also turn off readers and lead them to avoid spending time on clickbait-filled sites. Optimizing for user clicks resulted in drops in user engagement and trust.<br>The lesson here is general. Optimizing for one metric often comes at the cost of a separate quantity. Make sure that the quantity you wish to optimize for is indeed the “right” quantity. Isn’t it interesting how machine learning still seems to require human judgment at its core?</div>

# Binary Classification Metrics

Before introducing metrics for binary classification models, we think you will find it useful to learn about some auxiliary quantities. When a binary classifier makes predictions on a set of datapoints, you can split all these predictions into one of four categories (Table 5-1).

_Table 5-1. Prediction categories_

| Category | Meaning |
|------------|-------------------------|
| True Positive (TP)  | Predicted true, Label true |
| False Positive (FP) | Predicted true, Label false |
| True Negative (TN)  | Predicted false, Label false |
| False Negative (FN) | Predicted false, Label true |

We will also find it useful to introduce the notation shown in Table 5-2.

_Table 5-2. Positives and negatives_

| Category | Meaning |
|----------|---------------------------|
| P | Number of positive labels |
| N | Number of negative labels |

In general, minimizing the number of false positives and false negatives is highly desirable. However, for any given dataset, it is often not possible to minimize both false positives and false negatives due to limitations in the signal present. Consequently, there are a variety of metrics that provide various trade-offs between false positives and false negatives. These trade-offs can be quite important for applications. Suppose you are designing a medical diagnostic for breast cancer. Then a false positive would be to mark a healthy patient as having breast cancer. A false negative would be to mark a breast cancer sufferer as not having the disease. Neither of these outcomes is desirable, and designing the correct balance is a tricky question in bioethics.

We will now show you a number of different metrics that balance false positives and false negatives in different ratios (Table 5-3). Each of these ratios optimizes for a different balance, and we will dig into some of these in more detail.

_Table 5-3. Binary metrics table_

| Metric | Definition |
|---------------------|--------------------------|
| Accuracy                  | (TP + TN)/(P + N)
| Precision                 | TP/(TP + FP)
| Recall                    | TP/(TP + FN) = TP/P
| Specificity               | TN/(FP + TN) = TN/N
| False Positive Rate (FPR) | FP/(FP + TN) = FP/N
| False Negative Rate (FNR) | FN/(TP + FN) = FN/P

Accuracy is the simplest metric. It simply counts the fraction of predictions that were made correctly by the classifier. In straightforward applications, accuracy should be the first go-to metric for a practitioner. After accuracy, precision and recall are the most commonly measured metrics. Precision simply measures what fraction of the datapoints predicted positive were actually positive. Recall in its turn measures the fraction of positive labeled datapoints that the classifier labeled positive. Specificity measures the fraction of datapoints labeled negative that were correctly classified. The false positive rate measures the fraction of datapoints labeled negative that were misclassified as positive. False negative rate is the fraction of datapoints labeled positive that were falsely labeled as negatives.

These metrics all emphasize different aspects of a classifier’s performance. They can also be useful in constructing some more sophisticated measurements of a binary classifier’s performance. For example, suppose that your binary classifier outputs class probabilities, and not just raw predictions. Then, there rises the question of choosing a cutoff. That is, at what probability of positive do you label the output as actually positive? The most common answer is 0.5, but by choosing higher or lower cutoffs, it is often possible to manually vary the balance between precision, recall, FPR, and TPR. These trade-offs are often represented graphically.

The receiver operator curve (ROC) plots the trade-off between the true positive rate and the false positive rate as the cutoff probability is varied (see Figure 5-1).

> **Figure 5-1: _The receiver operator curve (ROC)._**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAASEAAAEXCAIAAABtXGX8AAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u2df2wb53nHn8axxUtly1Jjy95qq+2JsSZyqhbDEZc5dpliPmsdE7Z03P1QgmyUBay4tvFSFgKcdH+4AbQQmYvmBhSOOKQJkNZK2NJhAfkMJMo0z2DU2o09UqEjErXkAJEUTxptwUfJULA/3uPxSFH8eXe8I5/PH4Z0Mu/ee+/98n2e5573eaG7uxsQBFGH7u7uzwGAIAjYFwiiBhRF3YO9gCCqghpDENQYgqDGEARBjSEIagxBUGMIgqDGEAQ1hiCoMQRBUGMIghpDEAQ1hiCoMQRBjSEIghpDENQYgqDGEAQpnXsN0cqenp6rV6/i00KqSFdX1/vvv1/GB41Rz4OiKCw6ghhxEGI9DwSpGX8sztkplk//zrMUoeBBBEGNFYRnKasnlHHAGfGGBUEIeyPOlKByHkQQ1FhhfVHOiDfgtcmOBX3uQZYGALrXZfMF+XUPIojhUT2uyHCCwAHEOX/abpyKADjIz7TZAv6pODCQ6yCNDwhRkfmF5djM7eTy6uUPF5dXPrs0ufDpwvLUzO3r5x2mTRsMo7Hc2DraizwoaZVhxsfH5UcOHDjA8zjbIQAAlyYXl1dWpV9vLd0NxxLy/3Dxg5vkh8TS3Ujmn9byM9+vn/mnIwbXWCgaA6CLOSjZlygnQyGNaTk3Zu/cmL2z3kcKjv78Hy8V06YND3Y2A8CuHfft2nHftuYGc9vme1YXn/57xxcaTxjJVswBbbZAVGY3Whw0AOQ8iFSH5Mrq5cnFzIliYXnlM+nX2MzS/EKyeHmojaW9qalxY1o/DRse/JNm6deGTffs7WwhP7fv3ry9pWG985w/f/nEiRNPPvmkwTUGjMPtHOKOMywdH/WH3IPMugcRVTyQLGtKmnMuTy4mZRZXJTzcff/ag9tbTO27G9f7SFPjRkt7U55zbmtpMO/erGr/HDp0yHgxj3UCIYEgZaU8AOAOCEyeg0gZEP0QV16SUDiWuLV0t5iPb29paM8cylmzhHxaKGZy0D/T09PBYJBlWTVOjrlUhvf1P11IhmMJYrwV9Fik6WVvZ0vDpnsAoH134/YWk+SW1GEfTk9PMwxT0EQsO5fqXhymxjLzIrHE/MLypcmFPHIy7968raWBGGbEAGvYtGFvZzP2YdkCM56tiBTD1Mzty5OLlz9cnJq+nTNMBwB7O5t37fh8++5Ga3vTlsaN5F/suuJJJBKqCgxtRX1BonkXP7h5+cPF30UWstwn4iZZ2pu2tzTs7WypW9POWIMQbUVdTFYXf3/z8oeLlycXp2ZuZ4nqwc6WvZ3NeztbHuxsVjDzAJmenh4eHj558qQG1zKMxqQ8j9pI77j4wc3RC5+8NzGfpauHu+/f29liad9CZioUg3o+WF9fnzaXQ1tRO24t3X13Yu7chdl3J+YkO3BL48ZHH2rd29lMpiwUgGYCe+6559BWrBFuzN45d+GTsd/Ov/v+nHTQvHvz1x7a3rt/Z853tYiqsCyr0qswnMe05sy5mdfe/sMlWVLSoz2t9n3bD+/fiXYgxjyQiiau196+fubc9PzCMrEGe/fvtD+0/fD+nRi3qKKJ+NJLL7344osmk0njS6PGlOTd9+dG+BuBdz8mv+7tbH7qsS8//ugfo7R04oNpLzDUmGLBjF+/8zH3iymSeGHatOHbvbv7XV9RO4EVUTXIgRrTBcmV1Vfeir/8xhSJE5p3b+53feWbX/8iJlvoCo2DHFlgzKN8Au9+/CPuf4jT9e3Du5967MsYfK9hMOahKZcmF3/EXSUBw4e77z/J/mn+hU9IVUzEl1566ac//WnVW2KYGqYMw5DaiwxTzaVlN2bvPP3c+9/4zn9emlzcteO+V3/c86uf7EeB6dMH27dvnx4aU4qtyLOU0yc/4A4InCYDXg+24q2luy+/MfXKW/HkyuqWxo2epzuOHcFyCPoVmOLZ9CrX4iYFfIMOIYNwx1C9FPWNxBLf+M74y298lFxZPXaE/t0vD6HAdMvWrVu9Xq+qy1UUn8d41j51fIxdt2IUywKn7nxW3Xns52f/8C//Hk6urO7tbP7Zj/ZhigbGPEr6FMYV85FcWX1m6PfknfJ3/+6BEwOdONT0bCJ6PJ6RkRG9aaz4mAfP1tlmD7eW7rqeuRB49+MtjRvf+Nc/R4Hp3wdzOBw6bFvxGmM4QRAcQXFnFcrOxWv5md2YvfPNZy5cmlw07978zrD90Z5WHMf1FuTQ0h/LSZyzi3uxaBBd1NhWvDF751vPXLgxe2dvZ/MvXnwYkzZ0TjKZHB8fV6M0osa2YkpZ4jzWD8MkuhiA2trISBLYoz2t/p/sR4HpH5PJpLbAtLEVeVauLEGKMzKcJi/JBgYGPB6P2rtC31q6+/Rz7xOB/cfJhzBfXv8mYjKZ1Hk7S5rH3IGMCD7PahgDOXHiRFtb27Fjx9TL80iurP7tDy9GYglLexMKzBACq9ZyFcX9sTX5HRI2b3j992aG88e+P3T5zLmZXTvue2fYjiainpmbmzt48KDGQQ4N3o/xLBV0aJQ7VQWNnTk38/2hy1saN74zbMe3zPpnfHz8wIED+h+E+A5aZGrm9l/2v5dcWX31xz2H9+/EEYwoqLFi/DHid5GcxSxqIaJ4a+nuPz43QRIRUWA698F6enrm5uaM1exiNEYihwwnrKVc0zH9DkCuUknGmkr3hVcmp2Zu7+1sxkwO/Qc5HA5Ha2tr7WlMcXjW6neFBUEQwt6IM5UxwrPOiDcsHtRKZZcmF39+9g+mTRte/OduDCTqOchR3Zoc2miMZ5XKoIpPRWyuXhoAgGYH3aFoDACAD/rcgywNAHSvy+YLaiCy5MrqD//tAwA4doTGdZZ6prW11ev1GlFgUHK+4jD0V+6L0WZLyD8aJzbjkM/W0U6EJ/8PEJlSPx/ylbfikVhie0vDs0/vwXGsc/SZ76uCrUizY5Iv5giWqzOGEwajVoqiKKvflX7DRsSmFfMLyy+9eg0A0ErUeZAjkUgY+i7Kzlcky6LLinnEOTs11BEWBEEQBqNWSaii0biOLlP1PCQqTPggi5oP79+JsUQ9BzlYlm1qMrYZX3xdKp6lnD6bNywIlSZ2xKIhm2uYnIVxuGFoKg4MbbZANO2wgcWReRll90OaX1h+/e3rAOB5ugNHsw5JJBJ6Xq6ipj+mSOZUe4ct5Y8BH/SBxUwTtfmGuDgAxEf9IbdD1YQSaRLDUIc+aWpqGhkZqQGBFTmPkSwqRzBH1mJZa8dodiwQpayUB4CkPDKiiANB8ag7IKgoMZzEDEFXV1dt3Eg95lK9/MZHL5yePLx/56s/7sGhrDcf7PHHHz9z5syePXtqYxCWWs8jK4qo6doWBXnlrTgADGDxNl0GOY4cOaJDgWlgK4pWoo/KtBZt3uNGu+FzFz6ZX1i2tDfhBpa6IplMGjeTo3KNMZwgcFVd26IgrwWvA8DfHN6Nw1pXmEymkZGRmvHB6tcfm5q5/chT75g2bYj+5q/wvbN+ZjD9r2UGXNtS7CT29nUAePKxL6HA9OODdXd3q12mxRC2IgMAjCBw1WsowzDj4+MAcODAgfLeRydXVs+cm0FDUW9Bjr6+vpo0EUvSmC6oPM/jzOjMraW7ezub8b0zBjm0pIR8xThnJ2tbpKRFY1mKJNqB+63oJ8hx9uzZmhdYSRrjT3nAO8zS5IewtkspK+bS5CJZxoIZwHqYwcgPNfYeTIF5DMBipsmKL3GFpYEge698+3AbRjv0EOSYnp6un1su3h9jjnuHrBQFYPOGWZpnKadP3bRCRTl34RMAsD+0HUd5dQVGsunb2trq567r4v0YeS22pXHjR7/5Bg70KpqINpvt2WefNWg2fdnvx+6th6dLJrFe9MSqislk4nnecFWltPXHcryFNkbMY2xiHjVWRaRiAXUoMCitLpXT5w4oVF9RQ+YXli9+cNO0acPX0Bmrkg/W09NT25kcys1joPLi5LxI9TxKLeMxNjEHAA//2f0YUayKwEiQo7YzOZTSWLoWQFXgeZ5MnaUmfIxe+AQAmId34IjXnqNHj9ZGTY5KKKFmTtAHIRArBEgzW0Df5mJyZfU94ow98kc44rXn/PnzRq8qpaXGyCoyg3Hx9zeTK6t7O5u3tzTgiNeMubk5Et5AgUGV6t1raGFenAUA+0Ot+KS19MEOHjwYDAaxK8rQWCoZ2M7FgWeVKX6v+jwGGLXXVmAkm964pbOrqLE4Z7f6XWEh7LUR0zFg8fTrWmW3lu5Ozdze0rgRF7NoxsDAQD0sV1HJH4tFQ+7BMRokVTEOtzMYA9BtevDFD24CgBUFpiEjIyPog5U9j7V32DJi9+ktV/TKf39wEwCw/pQGzM3NkRUrKLBKNEazY2GX30pZPaGQx5q15YouicQSqDHNghxvvvkmdkWFGoPMnZEUqn1fNKXmeSRXVi9PLgLAg53N+Jg1CHLU+YtmRfyxKlNqesflycXkyqqlvQlTqDDIYYh5jGelJPvc26WjoVi/QQ4UmAIaE3dD5xiAONfvsQQEQfflPEjA4y9QYxjkMILG0ruhy7cGo9lBtyY7o5cHCdw/2NmCz1ilIAcpd4koG/OA+Kg/JMXr5bukl+5epdZ7ypNFpIOVzo9TM7dvLd3dteM+TFNUKchx4sSJQ4cOYW8opTHGIU5Y8VF/SKpJxZ/ylL3bpWh8CoIgSxaRDlZuhZKIIjpjauDxeHC5SkkUFVdkuECQoigQa1KJuyWVva4lzg353INkW2mxznfKIhVoAKB7XTZPkOfK31M9HEsAZnioFuTATlDDVmS4jHdiDFdJnYFYNGTrmGIzbUW55UmbLRCZqiAVMn5jCQDoXY34gBU0EaWyHIjyMQ82r+VW4M85CXmiDiHLVgTlMrPCU/8HAFbzVnzACvpgOIOp6I9xjmCuSIT4pizoKGNGS3lyjMMd8o8SkYWisTyNSOV5SKyX8HFr6e78wvKWxo0Y8FBQYH19fceOHcPeUMsfS++kSTkzhBIQyikU3N5hg2j2QdpskQ7GpyJgcWRmahWf5zE1s4TOmIK88MILmMmhgT+W4ZRVWvmN7nWlUvhl1fPTNXlkL+HKNxRxzZhSnD59GgWm/jymMDQ7FohSpPyOOyCIucUMFwimD5ZfiSd2YwkA2jHgUbGJCAB1VZheB/OYkkhzonwuzHmwZEhQEQMelftgExMT2BXG1ZiK/C6yAADm3TiPVSSwEydOPPHEE9gbqLFs5heWby3d3d7SsKVxIz7d8hgeHsZMjmppzAB1qfDNWOWcPHkSBVYVjRmjLhVmeFRiItbzzg960FgsGkqtcBEjFA533rfGVQGDipX4YFeuXMGuqKLGqlyXqsh6Hjdm7wCAuW0zPtpSBYY+mEqUtFdtnLNbPaHUbzavZnWpit8m9JGn3pmauf3bXx7ateM+fLpF8vzzzz/wwAMoMKUGYdanam0/6C8dCiZXVmffc+KYQHSisVL20cxOCl57pMrMLywnV1ZxBsMgh+H8MVICwOkDnzMj8d2ptzrB6IxhkEOHFJOvKGXdBx363tEvNnMbAHAewyCH4TQmKU3vG6yTeeyLrRQ+1/wEg0EUmA41lhVVJOhrr1qiMZzHCsKyLHaCrvwx0Ss75QFvWBACbnAHBEEIe21QwSovNTX2eXyu65mIWBRRzxoDAIuZTleC02ENU5zHCvpgqDE9a6y9w0ZqRaV+qKiGackUzPNIrqzemL2DZTzyCAxLBujcH6PZYZfdypoFjh20kPXKNm9YM1uxYD0PnMTyMDExgQIzQMwDaHaMvOYmwXyd8enCMmpsPXC1pSFsRd1ldeScx1q/YMKHKjcRcXtLI/pjOmV+IQkA25rRGcvwwa5du4ZdYRSNxaJkH+gMdDSzzaOtuEZg6IMZyx/TpRMm49bSXQBowjIeAAAwMTHR39//gx/8ALvCQBrTO8Qf29aC/hgABjmMaSvqHRJXrPOXYxjkQI2pyNxCss79MeKDkW2aEYNqrJq13/LneSRXVklZxToXGGbTG1pjVa79xvM8qdOdM+GDGIpNjZvq9kFeu3YNBaZPio95xKIh9+AYDZKqGIfbGYwB0Dq4DRK431bH8xjugF4D81iVa7/l59N6dcamp6c5jsNxXBsao9mxsMtvpawe8i7a6ndpVvutcMDjf5MAsL3OAvfEB2tqwp3WasRWJDIT9LmA9tPFugvcY5Cj9uYxNcjKMyYFsMpJ0SIaq6u9WhKJBAqsxjTGs2LUXkGFOX2Zv0a8YUEQwt6Is0SVJZdXoc4Sgru6ulBgNaYxhhMEYRj6lcoH5lknuN2y34M+cc8KutdlK7GKAUmkMjVsqAcT8fnnn8eBW7u2Is2OSTuuO4IV6IxnnRDgHOkD8soFtNkCpa2jSSzdhTqIKxIfrKEB1+/UsMZSmR4URVFBR9kbN8e5oYj3ePZn874JkPI8JOQJHyTpvmHThpoXGC5XMRzFxxV5lnL6bN6wIFQar49z/X7X8BgNkLl7WSia5412/noedZIQzLIslkasYY0ptn4sPuoPhUJWykN+9VERb3iMpc0WiMrsRoujaCknV1aTK6s1H1Rsa2tDgdWqrUgi7FJgvdJ10DKfjtRDJW+yGYdbzCOJj/pDpVRHTSUr1qbGpqenv/e972E2fW3PY2Kle0blddAMFwiSmnLgDpRSW584YzVpKEo+mMmEa09rP+ahxv5jDJcRNWE4cX4rLZSSqNGABwY56mce41lKfFnso3wZf7F5j+vgHmq4eikGOerHVtT1/mMkyaP2XkBjkKPebMUsu05HkGTFmkmkIkEOHJp1pTGF44qKs7zyGQA0bLqnNgTGMMy+fftwaNabrahFXLFAI1L7+hw4cCDrfTSpEFwDi8dwuUrdakwXFNy3pQbYunWr1+t1OBw4LuvTH4M4ZydrW6SkRZ1U4q6ZuGJTUxMKrJ41xp/ygHeYpaVNa8tY5oWsYyIePXoUMzlQY0D2qo1zQz6bq5fW0T0YOpeK+GBdXV2YyYH+GHPcO2SlKACbN8zSPEs5faWlPKlGcmUVjFloADM56oHPAYAgCDpvJUVReRrZ9a3R+YXlq7/qNVzKYjKZDAaDuP+DIcg/CPN8qhbeKc0bdvGYyWRCgaE/Jke+CprSttx9TUFMxEQigV2BGssSmNXvCkvlPMIuv1UHMjNc4J4I7JFHHsHao6ixLGLRkFg3ikCzg+5QNKZVQ/Pv22IU5ubmMMiBGluPKte7X2/fFmM5Y62trV6vFwVWVxQfu6fZsTDYpTIcADavHurdL6+sgqEWaGImB85j+WUmpNHNhhKG8MF6enowyIEaMySkmIeeX0CTIAfLshjkQI2tjyxor7cMRVLMQ7eJVCTIgctVUGOFFNbvsQRIwB7zgEujtbX19OnTKDDUWH5iUancId3rspVWiR5txQMHDuA4Q42VhIYvxQxrK2KQA6lEY0hhgWGQAylVYz6nGPKwekLpX7SMgKyX56G3eSyRSGCQA5FTfH3FKrNePQ+9+WNNTU0jIyNdXV04thC0FdUCBYbUlMZSxe6rfCPT09Pd3d3Xrl3DIYXUmsZSm7aYqiswhmGOHDmyZ88eHFJIJRpLpXvYuTjwLC7SBACAZDKJy1UQRTSWWqMZ9toAABguYPH0l6uydGFveWRSOmqkVBKTyTQyMoICQyrXWPYaTWAcZa7RjHN2Z8RLVlQHwJmaDnlWPFpSwlYV10FLFRExyIEoojHl1mjK1ZoWKh/0iUfpXpfNF9T5VEaCHFevXsUxhOSnzDWaVspT/hpNcY8KEJVl6zgOZJ91EFcv0mYL+KfiwND6FRjxwXAGQxTUGJGZoOieczzr9LkDgiilvLOitG+LBNnAhRQJ3qZhrQEMciDqaUxR4pzd6rEEZBsHhqIxgPWmrvXyPEiRYJOGtQZMJtPZs2cxTI8o7o8puscfz1JWvyssExhttqTlNxUBi1l/hqIU5ECBIWpojOGEDMJem817vJw6bGJcMcuZYxxuMaYSH/VLC9byo2WyIglyTE9P46BBVNLYWuds0OI5VcY8Fh/1hyDksWZPhgwXsHisFEVZPZZAcVtPa5Z0L+1w2dbWhoMG0cof44M+KRKoTOREFwn+OU3Exx9/HJerIGprjGcppy/zkE72RtIgyMHzfGtrKw4XRFWN6XGSUTvJI5FIkLXMKDBEA3+MZ6n6qkhFanJgJgeimcbaO3RWkUptgZEgB2ZyIJppLBYNyYOBuqjnoV7s/ujRoxjkQOrLH8uZ56Fe7P78+fNYVQrRbB6rI09sbm6O/IACQ7S3FevCBzt48GBW5jGC1LXGFIzdS0EOLJ2NVEVjsrKlVYp5qM3AwAAGORA1KDLm4ZavQdEPyyufKXWqs2fPmkwmHBAI2ooZzC8kK7QV5+bmyIoVFBiCGlMeEuR48803cRwg1bUV5fU39MWni8sAsK25oTyBkZIB6IMhOI8BrJPnkVxeBQBTQzmFBgYGBrAmB6KTeUwXKJ7nMTIygi+aEZzHClBGvqIU5ECBIaixwiyvrJbqg2EmB4IaK4H5hWUoOnYvZXIcOnQIHzyCGlMej8eDmRyI9txr3KaXNIkBwMjICD5vBOcx5Z2x6enpRCKBTxpBjZXJ9ryV7okPhjMYghorB7KwpWnzpvwC6+vrO3bsGD5pBDVWgLV5HiQhOE8i1QsvvICZHEjVMXCex6eFYh6nT5/GB4zgPFY+JK64dh6bnp7GnR8Q1JgiGksCQOsXTGt9sGAwiI8WQY1VCol5bGsxZQmsr6+PZVl8tAhqTBmNyf0xDHIgOkRXMQ9pa5iiyofcmL1j2rRB/n4MgxwIzmP5FeaMeMOCIIS9EWehgldTM7elSQyDHAhqrCiJBX3uQZYGALrXZfMF84ssPrMEAPTuRuKDTUxM4LNEUGP5RTMVkX6mzRYosEfM4f07/+u1r//DX28ly1WeeOIJfJYIaqwQto729f8o5Xmk66ceO/KD7/ZVZbnKwMBAVXoIr2vE634OAARB0ME8xtmt0UEx1MGz1FBHeIylxT9SFJWzkesdV5W5ubmenp7r16/XyXWr1c96u255jaEoSjfzGG22ZNiNFjOt06n/ypUrX/3qV+vnukjN2IqMw+0b4uIAEB/1h9yOyks65p/x8/w1/wc/+uijBx54QI3r5v9rta5boVml0nWrdb/G9scYLmDxWCmKsnosASWq67/++uvl/TX/BwuO9bKvm/+v+a87Pj6e/wVGJa0q+4MVtqrsS6t6XSPHPIDhBAKn07rEOrYV33rrrSNHjuitr6rVKl31hrHXQbe1tanx1/wfTCaTe/bs0b5V+a975cqVxx57TG99Va1WqXrdUtFNXDEvO3bswJocSHVpamqanZ0t9VMURRlDYwhiUPQUu0eQGgU1hiCoMQRBjSEIghpDENRYCfBsKvme1+xSGdfSrAE8W4XrSlexc3GtLh3n7Nr3c5yzF75cpW24x5gKK2HFdKXPQLyUIATAmRpymjWAZ0nxBU1vXLqKELB4+rW5ZZ61+l1hQTy/Nv3Ms5TVEyrUvZW3wYgaK23FdEXEoiHxUgCMwx2KxrRsAM86we3W+Mbj3FDqKsBwgrjASOVLx6ciNlcvDQBAs4Ma9DPPUpQz4g14bQW6V4E2GFBjJa6YrgiGS+dO8kEfWUWqUQN41gkBzqH1jceiIVvHFJtpK6p9adpsCflH4ymNq9/PDCcIwhjbXmhcKdEGY/pjeVdMq2e1SVOaBg2Ic0MR73GmKjce8kQdQpatqPKlGU4YjFopiqKsfld6da7GDzrn5SpugzE1JtoSms2cnJ1ygrwendoNiHP9ftcwS1fpxlOr9xiHOzW9qHvpOGenhjqI3zsYtUrRBY0fdM7LVdwGA2pM4xXTPEtZ/a6wTGDqNyA+6g+FPFaKoiinD3xOYrNpc+PtHTbt+zwWDaX8MWAcbmKSafygc15OiTYYcR5TfsV0vu9XZ8QrqyyiTQNodkxcSScE3OAOiJEHTW6c7nXZUlfhhnzi0Ff50u0dNmnC5IM+cShr+KDXfawKtMGQ+0EzXCBIWSkPALgDgpoSG/WHIBQilwKQChhr1oBq3DjNjgWi6aukoqqqXlp+TbB5w2OMtg86T/dW3gZc24IgKoJrWxBEdVBjCIIaQxDUGIIgqDEEQY0hCGoMQRDUGIKgxhAENWYw0svUqRxr5LP/Z+lrWdecf73TQ/6Lrv1B+X4o9rRxzp5xF4qsus/oKCVuMN0q8dR2Lp7vNovp4ew7R40VizsgyBhjaRXPH3b5rSWMIJody9olY+0Rzb+VMtbOKLPqPl0vIKs0Q/lIHRUf9YfcAfJg8/ReMR1Ls8Muf7+GKqtpW1H2tbp2SEh/lI2EIquj0L0um7ggNlell+wzi9+pPGv1hMDnpFg+9S2bURAn/cs6zYhzdjvL2lPHs+9Odv5C98Kf8lhS600VW3UvqxcAkFGngKVYLtVYWWtytTB9U3Yuvrbr5AeL+f8sy8qeLy/+QrODFs8pHjWmxHe1xxIgZVhSqzXW/jHsBbG7ZfVxCnwH86c8IYuZJsM6dYlUpZccZ04NurDXlsrcFw853NJY5YM+t4Mp0IyQDwbJ3lFr705+/vz3krpUSgvKrLqnzZaQx5rbjvZ5ooNCxgyYs4Xp/sxYgp2+tSz7pOD/5zjZKtP0l0RGx6PGisXnzPYE0nYD3euyZa1mpc0W8AX5bGsk/QikJ5Pj/M6IN8wx5JtbrAdAs4PiR9aeOQ/pZy1JLH8zJG3kvbv8J4lPRQquny9n1T3DCULALa4tzXRa5d3kC/LrtFDWn7JpMM+3aBH/nznuBXL7Gd8s7R02NevA1L4/lh7ZojmS6WyIzyTgFnWTNlUkIWVUXFvr74lPMxaVnVZaP5zzzIVEljEA8jQjywFa7+7yniSj2etQ3Kr7tbUnpW0ahYDFU8BpXdPCYhpW6o0A3esC/2g8s4cxrqgMPEulakSEvbbcX7uCIARkVoNcSEWETTKW5X8TLIIAAAF9SURBVMseeI4zFxBZ5gAophmF7i7PSXJWE8ie5OV247qr7tOKWjthM8e9OaYKucW5poUFG5av/wuIjKuaxGpYY/GpCIjWCH8q+5teFhNOPXTZGvtig+C02RKSnDlxWX6OMxcWmdMpDYAim5H37gqchDZbCtSBKWvVPc9mTtyxaEiqfiF1U+qTOVso68+iHkGR/5/udYHHkykxeePU5t6a1RjNDropJ+UDAHcg4PYFs+K3XruVosSvU9FXSK93Ty93z68PLiw/DUsDQI4zy4e+x0lBINyROaDBB9IAKK4ZOe9OOr/A5T0J43A7gzzHMHlurPRV9/LOEO9emt1sbhiiKKesMTlvM+MU7oDAADdUbP9n/n9ZVzBA97psfpe8kB4f9LkdnEYjEWsN1CVxzt4Pw8q/RVzPrB3qCGt0sWLuVsu7x1oD9Yrm72Gr6pVb028DYf3alTiPIYgRwXkMQVQHNYYgqDEEQY0hCIIaQxDUGILUJFu3bsVOQBD19PX/etMAxk+peUUAAAAASUVORK5CYII=" />

The area under curve (AUC) for the receiver operator curve (ROC-AUC) is a commonly measured metric. The ROC-AUC metric is useful since it provides a global picture of the binary classifier for all choices of cutoff. A perfect metric would have ROC-AUC 1.0 since the TPR would always be maximized. For comparison, a random classifier would have ROC-AUC 0.5. The ROC-AUC is often useful for imbalanced datasets, since the global view partially accounts for the imbalance in the dataset.

# Multiclass Classification Metrics

Many common machine learning tasks require models to output classification labels that aren’t just binary. The ImageNet challenge (ILSVRC) required entrants to build models that would recognize which of a thousand potential object classes were in provided images, for example. Or in a simpler example, perhaps you want to predict tomorrow’s weather, where provided classes are “sunny,” “rainy,” and “cloudy.” How do you measure the performance of such a model?

The simplest method is to use a straightforward generalization of accuracy that measures the fraction of datapoints correctly labeled (Table 5-4).

_Table 5-4. Multiclass classification metrics_

| Metric | Definition |
|--------|----------------------------|
| Accuracy | Num Correctly Labeled/Num Datapoints |

We note that there do exist multiclass generalizations of quantities like precision, recall, and ROC-AUC, and we encourage you to look into these definitions if interested. In practice, there’s a simpler visualization, the confusion matrix, which works well. For a multiclass problem with k classes, the confusion matrix is a k × k matrix. The (i, j)-th cell represents the number of datapoints labeled as class i with true label class j. Figure 5-2 illustrates a confusion matrix.

> **Figure 5-2: _The confusion matrix for a 10-way classifier._**<br><img width="300" src="data:image/png;base64, ..." />

Don’t underestimate the power of the human eye to catch systematic failure patterns from simple visualizations! Looking at the confusion matrix can provide quick understanding that dozens of more complex multiclass metrics might miss.

# Regression Metrics

You learned about regression metrics a few chapters ago. As a quick recap, the Pearson R2 and RMSE (root-mean-squared error) are good defaults.

We only briefly covered the mathematical definition of R2 previously, but will delve into it more now. Let xi represent predictions and yi represent labels. Let x and y represent the mean of the predicted values and the labels, respectively. Then the Pearson R (note the lack of square) is

$$ R = \frac{\sum_{i=1}^N(x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^N(x_i-\bar{x})^2}\sqrt{\sum_{i=1}^N(y_i-\bar{y})^2}} $$

This equation can be rewritten as

$$ R = \frac{cov(x,y)}{\sigma(x)\sigma(y)} $$

where cov represents the covariance and σ represents the standard deviation. Intuitively, the Pearson R measures the joint fluctuations of the predictions and labels from their means normalized by their respective ranges of fluctuations. If predictions and labels differ, these fluctuations will happen at different points and will tend to cancel, making R2 smaller. If predictions and labels tend to agree, the fluctuations will happen together and make R2 larger. We note that R2 is limited to a range between 0 and 1.

The RMSE measures the absolute quantity of the error between the predictions and the true quantities. It stands for root-mean-squared error, which is roughly analogous to the absolute value of the error between the true quantity and the predicted quantity. Mathematically, the RMSE is defined as follows (using the same notation as before):

$$ RMSE = \sqrt{\frac{\sum_{i=1}^N(x_i-y_i)^2}{N}} $$